# Archivo para generar el cruce con geopandas en databricks

In [0]:
#dbutils.fs.put("/FileStore/my-stuff/my-file.txt", "This is the actual text that will be saved to disk. Like a 'Hello world!' example")

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
from pyspark.sql.functions import year, month, dayofmonth, concat_ws, lpad
from pyspark.sql.functions import col
from pyspark.sql.functions import count, sum
import pandas as pd
#import seaborn as sns
import pyarrow.parquet as pq
from pyspark.sql import SQLContext


# SparkConf().set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0-alpha3")
sc = SparkSession.builder.getOrCreate()

sqlContext = SQLContext(sc)
sqlContext.setConf("spark.sql.shuffle.partitions", "auto")
sqlContext.setConf("spark.sql.execution.arrow.pyspark.enabled", "true")
#spark.conf.set("spark.sql.adaptive.shuffle.partitions", True)
#sqlContext = SQLContext(sc)

#df2 = sqlContext.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-01.parquet")

/databricks/spark/python/pyspark/sql/context.py:117: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
spark.conf.get("spark.sql.shuffle.partitions")

'auto'

In [0]:
#Función para el geojoin 
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, pandas_udf

def create_sjoin_udf(gdf_with_poly,join_column_name):
    def sjoin_settlement(x, y):
        gdf_temp = gpd.GeoDataFrame(data = [[x] for x in range(len(x))],geometry=gpd.points_from_xy(x,y),columns=['id'])
        settlement = gpd.sjoin(gdf_temp,gdf_with_poly[['borough','geometry']],how='left')#.fillna(np.nan)
        
        return settlement.groupby('id').agg({'borough':lambda x: list(x)}).reset_index().sort_values(by='id').loc[:,join_column_name].astype('str').str.strip('[]').str.strip("'")# if pd.isnull(sum(x)) == False else np.nan}).reset_index().sort_values(by='id').loc[:,join_column_name]
    return pandas_udf(sjoin_settlement, returnType=StringType())
sjoin_udf = create_sjoin_udf(zone_gdf,'borough')

In [0]:
import requests 
import geopandas as gpd
request = requests.get('https://raw.githubusercontent.com/AlberGonglius/taller-1-big-data/main/NYCTaxiZones.json')
geojson = request.json()
zone_gdf = gpd.GeoDataFrame.from_features(geojson['features'])

In [0]:
zone_gdf

,geometry,shape_area,objectid,shape_leng,location_id,zone,borough
0,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ...",0.0007823067885,1,0.116357453189,1,Newark Airport,EWR
1,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens
2,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ...",0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx
3,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ...",0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan
4,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ...",0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island
...,...,...,...,...,...,...,...
258,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ...",0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn
259,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ...",0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx
260,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ...",0.000422345326907,260,0.133514154636,260,Woodside,Queens
261,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ...",0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan


In [0]:
#se leen los primeros parquet que se saben tienen la misma esrtuctura y solo las cols que importan

df1 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-01.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df2 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-02.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df3 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-03.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df4 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-04.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df5 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-05.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df6 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-06.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df7 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-07.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df8 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-08.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df9 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-09.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df10 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-10.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df11 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-11.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')
df12 = spark.read.parquet("s3://taxisabd2023/yellow_tripdata_2009-12.parquet").select('Trip_Pickup_DateTime', 'Trip_Distance',  'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','Total_Amt')

df = df1.union(df2).union(df3).union(df4).union(df5).union(df6).union(df7).union(df8).union(df9).union(df10).union(df11).union(df12)

df.show()

+--------------------+------------------+------------------+---------+------------------+---------+---------+
|Trip_Pickup_DateTime|     Trip_Distance|         Start_Lon|Start_Lat|           End_Lon|  End_Lat|Total_Amt|
+--------------------+------------------+------------------+---------+------------------+---------+---------+
| 2009-01-04 02:52:00|              2.63|        -73.991957|40.721567|        -73.993803|40.695922|      9.4|
| 2009-01-04 03:31:00|              4.55|        -73.982102| 40.73629|         -73.95585| 40.76803|     14.6|
| 2009-01-03 15:43:00|             10.35|        -74.002587|40.739748|        -73.869983|40.770225|    28.44|
| 2009-01-01 20:52:58|               5.0|        -73.974267|40.790955|-73.99655799999998|40.731849|    18.45|
| 2009-01-24 16:18:23|               0.4|         -74.00158|40.719382|-74.00837799999998| 40.72035|      3.7|
| 2009-01-16 22:35:59|               1.2|        -73.989806|40.735006|        -73.985021|40.724494|      6.6|
| 2009-01-

In [0]:
#!pip install geopandas

In [0]:
#df_subset = df.limit(50)
df = df.withColumn('PU_zone',sjoin_udf(df.Start_Lon,df.Start_Lat ))
df = df.withColumn('DO_zone',sjoin_udf(df.End_Lon,df.End_Lat))
df.show()

+--------------------+------------------+------------------+---------+------------------+---------+---------+---------+---------+
|Trip_Pickup_DateTime|     Trip_Distance|         Start_Lon|Start_Lat|           End_Lon|  End_Lat|Total_Amt|  PU_zone|  DO_zone|
+--------------------+------------------+------------------+---------+------------------+---------+---------+---------+---------+
| 2009-01-04 02:52:00|              2.63|        -73.991957|40.721567|        -73.993803|40.695922|      9.4|Manhattan| Brooklyn|
| 2009-01-04 03:31:00|              4.55|        -73.982102| 40.73629|         -73.95585| 40.76803|     14.6|Manhattan|Manhattan|
| 2009-01-03 15:43:00|             10.35|        -74.002587|40.739748|        -73.869983|40.770225|    28.44|Manhattan|   Queens|
| 2009-01-01 20:52:58|               5.0|        -73.974267|40.790955|-73.99655799999998|40.731849|    18.45|Manhattan|Manhattan|
| 2009-01-24 16:18:23|               0.4|         -74.00158|40.719382|-74.00837799999998| 

In [0]:
#crea la col "pickup_date" que contiene el año-mes-dia de "tpep_pickup_datetime"

#se necesita independientemente cada una...
df = df.withColumn("pickup_year", year(df["Trip_Pickup_DateTime"]))
df = df.withColumn("pickup_month", month(df["Trip_Pickup_DateTime"]))
df = df.withColumn("pickup_day", dayofmonth(df["Trip_Pickup_DateTime"]))


# Muestra el DataFrame resultante con la nueva columna
df.show()

+--------------------+------------------+------------------+---------+------------------+---------+---------+---------+---------+-----------+------------+----------+
|Trip_Pickup_DateTime|     Trip_Distance|         Start_Lon|Start_Lat|           End_Lon|  End_Lat|Total_Amt|  PU_zone|  DO_zone|pickup_year|pickup_month|pickup_day|
+--------------------+------------------+------------------+---------+------------------+---------+---------+---------+---------+-----------+------------+----------+
| 2009-01-04 02:52:00|              2.63|        -73.991957|40.721567|        -73.993803|40.695922|      9.4|Manhattan| Brooklyn|       2009|           1|         4|
| 2009-01-04 03:31:00|              4.55|        -73.982102| 40.73629|         -73.95585| 40.76803|     14.6|Manhattan|Manhattan|       2009|           1|         4|
| 2009-01-03 15:43:00|             10.35|        -74.002587|40.739748|        -73.869983|40.770225|    28.44|Manhattan|   Queens|       2009|           1|         3|
| 20

In [0]:
#Repartición para ver si mejora desempeño
#df = df.repartition(30, 'pickup_day')


In [0]:
from pyspark.sql.functions import count, sum

agg_df = df.groupBy("pickup_year", "pickup_month", "PU_zone", "DO_zone" ).agg(count("*").alias("conteo"), sum("Total_Amt").alias("tot_amt"), sum("Trip_Distance").alias("distancia"))

#si quisiera redondear:
#agg_df = agg_df.withColumn("tot_amt", round(agg_df["tot_amt"], 2))


# Muestra el nuevo DataFrame
#agg_df.show()

In [0]:
display(agg_df)

pickup_year,pickup_month,PU_zone,DO_zone,conteo,tot_amt,distancia
2009,10,Manhattan,Bronx,47426,1253392.169999991,461773.04000000004
2009,10,Bronx,Brooklyn,112,3684.4599999999987,1383.17
2009,10,Manhattan,"Manhattan', 'Manhattan",1,10.2,1.8
2009,4,Queens,Manhattan,318718,1.0633461370008713E7,3590736.34999996
2009,4,Queens,EWR,341,33231.529999999984,9886.34
2009,4,Manhattan,EWR,14980,1015392.4599999976,262787.92999999993
2009,4,Brooklyn,Queens,11829,281128.3600000004,102700.9599999997
2009,4,EWR,nan,39,1345.2399999999998,103.96
2009,4,Bronx,nan,400,7055.010000000004,1818.8999999999996
2009,4,Staten Island,Bronx,1,83.2,31.25


In [ ]:
agg_df.write.parquet("/FileStore/2009.parquet")